In [1]:
cd "Model/Segmentacion"

/home/jupyter-pcarmona@alges.cl/Model/Segmentacion


In [3]:
ls

data.ipynb         unetmIoU-0.211.pt  unetmIoU-0.285.pt  unetmIoU-0.293.pt
data.py            unetmIoU-0.214.pt  unetmIoU-0.286.pt  unetmIoU-0.296.pt
optuna.db          unetmIoU-0.215.pt  unetmIoU-0.287.pt  unetmIoU-0.734.pt
__pycache__/       unetmIoU-0.218.pt  unetmIoU-0.288.pt  unetmIoU-0.735.pt
txt.txt            unetmIoU-0.277.pt  unetmIoU-0.289.pt  unetmIoU-0.737.pt
unetmIoU-0.204.pt  unetmIoU-0.283.pt  unetmIoU-0.290.pt  unetmIoU-0.739.pt
unetmIoU-0.209.pt  unetmIoU-0.284.pt  unetmIoU-0.291.pt  unetmIoU-0.741.pt


In [4]:
from data import *
from torch.utils.data import DataLoader
from torchvision import transforms


#datasets
BATCH_SIZE = 4
mode="baches"
IMAGE_PATH, MASK_PATH = getpaths(mode)
df=create_df(mode)
train_set = OperationDataset(IMAGE_PATH, MASK_PATH, df['id'].values, t_train)
val_set = OperationDataset(IMAGE_PATH, MASK_PATH, df['id'].values, t_val)

train_loader = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_set, batch_size=BATCH_SIZE, shuffle=True)

In [10]:
import optuna
import segmentation_models_pytorch as smp
def Objective(trial):
    ENCODER_NAME = 'resnet18'#'timm-regnetx_002'#trial.suggest_categorical('encoder',['resnet50','resnet18','timm-efficientnet-b1'])#'mobilenet_v2'  # 'mobilenet_v2', 'resnet50', 'resnet34'
    ENCODER_WEIGHTS = 'imagenet'  # None, 'imagenet', 'ssl', 'swsl'
    EPOCHS = 10
    ENCODER_DEPTH = 3
    POOLING = 'avg'  #  'avg', 'max'
    DROPOUT = 0.1
    CHANS=3
    def getCriterion(CRITERION):
        if CRITERION == 'CEL':
            criterion = nn.CrossEntropyLoss(ignore_index=0)
        elif CRITERION == 'DL':
            criterion = smp.losses.DiceLoss(mode='multiclass', ignore_index=0)
        #elif CRITERION == 'CEL+DL':  #definir bien
        #    criterion = smp.losses.JacardLoss('multiclass')
        #elif CRITERION == 'JC':
        #    criterion = smp.losses.JaccardLoss(mode='multiclass')  #RuntimeError: one_hot is only applicable to index tensor.
        elif CRITERION == 'LV':
            criterion = smp.losses.LovaszLoss(mode='multiclass', ignore_index=0)
        elif CRITERION == 'Focal':
            criterion = smp.losses.FocalLoss(mode='multiclass', ignore_index=0)
        elif CRITERION == 'Tversky':
            criterion = smp.losses.TverskyLoss(mode='multiclass',alpha=0.5,beta=0.5, ignore_index=0) 
        return criterion
    def getModel(model, ENCODER_NAME=ENCODER_NAME, ENCODER_WEIGHTS=ENCODER_WEIGHTS, N_CLASSES=N_CLASSES):
        if model == 'unet':
            return smp.Unet(encoder_name = ENCODER_NAME, encoder_weights = ENCODER_WEIGHTS, in_channels = CHANS, classes = N_CLASSES, activation = None)
        elif model == 'fpn':
            return smp.FPN(encoder_name = ENCODER_NAME, encoder_weights = ENCODER_WEIGHTS, in_channels = CHANS, classes = N_CLASSES, activation = None)
        elif model == 'linknet':
            return smp.Linknet(encoder_name = ENCODER_NAME, encoder_weights = ENCODER_WEIGHTS, in_channels = CHANS, classes = N_CLASSES, activation = None)
        elif model == 'unet++':
            return smp.UnetPlusPlus(encoder_name = ENCODER_NAME, encoder_weights = None, in_channels = CHANS, classes = N_CLASSES, activation = None)
        else:
            raise ValueError('model name is not correct')
    model=getModel('unet++')#trial.suggest_categorical('model',['unet++','fpn','linknet']))
    LR = 0.002#trial.suggest_float('lr', 1e-5, 1e-2)
    WEIGHT_DECAY = 0.004#trial.suggest_float('weight_decay', 1e-5, 1e-2)
    #criterion = trial.suggest_categorical('criterion', ['LV','DL','Focal','Tversky'])
    criterion = getCriterion('Tversky')#trial.suggest_categorical('criterion', ['LV','DL','Focal','Tversky']))
    
    # optimizer = torch.optim.Adam([dict(params=model.parameters(), lr=LR),])
    optimizer = torch.optim.AdamW(model.parameters(), lr=LR, weight_decay=WEIGHT_DECAY)

    sched = torch.optim.lr_scheduler.OneCycleLR(optimizer, LR, epochs=EPOCHS,
                                                steps_per_epoch=len(train_loader))

    history = fit(EPOCHS, model, train_loader, val_loader, criterion, optimizer, sched)
    return history['val_miou'][-1]
nt=60
study_name = 'optuna4'  # Unique identifier of the study.
storage_name = 'sqlite:///optuna.db'  # Storage in DB.
study = optuna.create_study(direction='maximize',load_if_exists=True, study_name=study_name, storage=storage_name, sampler=optuna.samplers.TPESampler(seed=42), pruner=optuna.pruners.MedianPruner(n_startup_trials=int(nt/3), n_warmup_steps=int(nt/3)))
study.optimize(Objective, n_trials=nt)


[I 2023-04-21 10:28:31,396] Using an existing study with name 'optuna4' instead of creating a new one.
/home/jupyter-pcarmona@alges.cl/.local/lib/python3.9/site-packages/torch/cuda/__init__.py:123: UserWarning: 
    Found GPU1 Tesla K40c which is of cuda capability 3.5.
    PyTorch no longer supports this GPU because it is too old.
    The minimum cuda capability supported by this library is 3.7.
    
  warnings.warn(old_gpu_warn % (d, name, major, minor, min_arch // 10, min_arch % 10))


  0%|          | 0/82 [00:00<?, ?it/s]

[W 2023-04-21 10:28:34,658] Trial 27 failed with parameters: {} because of the following error: RuntimeError('CUDA out of memory. Tried to allocate 64.00 MiB (GPU 0; 7.93 GiB total capacity; 2.46 GiB already allocated; 33.06 MiB free; 2.53 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF').
Traceback (most recent call last):
  File "/home/jupyter-pcarmona@alges.cl/.local/lib/python3.9/site-packages/optuna/study/_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
  File "/tmp/ipykernel_64792/2580510792.py", line 50, in Objective
    history = fit(EPOCHS, model, train_loader, val_loader, criterion, optimizer, sched)
  File "/home/jupyter-pcarmona@alges.cl/Model/Segmentacion/data.py", line 224, in fit
    output = model(image)
  File "/home/jupyter-pcarmona@alges.cl/.local/lib/python3.9/site-packages/torch/nn/modules/mod

RuntimeError: CUDA out of memory. Tried to allocate 64.00 MiB (GPU 0; 7.93 GiB total capacity; 2.46 GiB already allocated; 33.06 MiB free; 2.53 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF